In [1]:
get_ipython().run_line_magic('pip', 'install db-dtypes')

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [2]:
get_ipython().system('pip install keras')
get_ipython().system('pip install tensorflow')

In [3]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [4]:
import os
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt  
import statsmodels.api as sm
import itertools

In [5]:
BIGQUERY_PROJECT = 'ironhacks-data'

In [6]:
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [7]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data` 
"""

In [8]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
data = query_job.to_dataframe()

In [9]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data` 
"""

In [10]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
wagedata = query_job.to_dataframe()

In [11]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list` 
"""

In [12]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
prediction_list = query_job.to_dataframe()

In [13]:
import pandas as pd
import numpy as np
combined = pd.merge(data,wagedata,on=['uu_id','countyfips', 'tract','tract_name'],how = 'left')

In [14]:
wagedata[wagedata['uu_id']=='a5c6dcff737e183f7931b472f10c3235']

,uu_id,countyfips,tract,tract_name,average_wage


In [15]:
combined['average_wage'].fillna(combined['average_wage'].mean(),inplace = True)
## Using the mean for the missing wage

In [16]:
firstgroup = ['edu_8th_or_less','edu_grades_9_11','edu_hs_grad_equiv','edu_post_hs','edu_unknown']
secondgroup = ['gender_female','gender_male','gender_na']
thirdgroup = ['race_amerindian','race_asian','race_black','race_noanswer','race_hawaiiannative','race_other','race_white']
## columns to impute

In [17]:
complete = combined.dropna(axis = 0)
#using the complete rows to estimate the ratio of each catagory occupying the claims
prob = {}
for i in firstgroup:
    prob[i] = (complete[i]/complete['total_claims']).mean()
for i in secondgroup:
    prob[i] = (complete[i]/complete['total_claims']).mean()
for i in thirdgroup:
    prob[i] = (complete[i]/complete['total_claims']).mean()

In [18]:
prediction_list['total_claims'] = 0

In [19]:
def approximatedimputation(data):
    firstgroup = ['edu_8th_or_less','edu_grades_9_11','edu_hs_grad_equiv','edu_post_hs','edu_unknown']
    secondgroup = ['gender_female','gender_male','gender_na']
    thirdgroup = ['race_amerindian','race_asian','race_black','race_noanswer','race_hawaiiannative','race_other','race_white']
    for i in firstgroup:
        for j in range(data.shape[0]):
            # if is nan, we use the approximation method to try to impute
            if pd.isna(data[i].values[j]):
                data[i].values[j] = int(prob[i]*data['total_claims'].values[j])
    for i in secondgroup:
        for j in range(data.shape[0]):
            if pd.isna(data[i].values[j]):
                data[i].values[j] = int(prob[i]*data['total_claims'].values[j])
    for i in thirdgroup:
        for j in range(data.shape[0]):
            if pd.isna(data[i].values[j]):
                data[i].values[j] = int(prob[i]*data['total_claims'].values[j])

In [20]:
approximatedimputation(combined)
combined.to_csv('submission_prediction_output(1).csv',index = False)

In [21]:
prediction_list.loc[:,['uu_id','total_claims','week_number']].to_csv('submission_prediction_output(2).csv',index = False)